In [30]:
import pandas as pd
import re
import numpy as np
from neo4j import __version__ as neo4j_version
from neo4j import GraphDatabase
import csv

In [31]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="USER1", pwd="password")

In [32]:
df = pd.read_csv('keyword_data_for_class_node.csv')
df_nic=pd.read_csv('NIC.csv')

In [33]:
df


,Name_of_class,Keywords_from_description
0,"'Growing of cereals (except rice), leguminous ...","['often combined within agricultural units', '..."
1,'Growing of rice',"['including organic farming', 'genetically mod..."
2,"'Growing of vegetables and melons, roots and t...","[""see 0128 '>"", 'capsicum spp .)', 'see 0130 g..."
3,'Growing of sugar cane',"[""see 0113 '>"", ""description =' growing"", 'sug..."
4,'Growing of tobacco',"[""description =' growing"", 'class includes gro..."
...,...,...
409,'other personal service activities n.e.c.',"['personal service activities n', 'record n', ..."
410,' activities of households as employers of dom...,"[""description =' activities"", 'class includes ..."
411,' undifferentiated goods-producing activities ...,"[""description =' undifferentiated goods"", 'act..."
412,' undifferentiated service-producing activitie...,"[""description =' undifferentiated service"", 'a..."


In [34]:
df.loc[0,"Name_of_class"]


"'Growing of cereals (except rice), leguminous crops and oil seeds'"

In [35]:
df_placeholder=pd.read_csv("Class_place_holder_name.csv")
df_placeholder

,Placeholder_of_class,Name
0,111,"Growing of cereals (except rice), leguminous c..."
1,112,Growing of rice
2,113,"Growing of vegetables and melons, roots and tu..."
3,114,Growing of sugar cane
4,115,Growing of tobacco
...,...,...
410,9609,other personal service activities n.e.c.
411,9700,activities of households as employers of dome...
412,9810,undifferentiated goods-producing activities o...
413,9820,undifferentiated service-producing activities...


In [94]:
location=df_placeholder.index[df_placeholder["Placeholder_of_class"]==9609]
re=r"a-z"
name=df_placeholder.loc[location,"Name"]
name=str(list(name))
name=name[2:-2]
print(name)

other personal service activities n.e.c.


In [20]:
df_placeholder
filter= df_placeholder["Placeholder_of_class"]==111
location=df_placeholder.where(filter)
print(location)


     Placeholder_of_class                                               Name
0                   111.0  Growing of cereals (except rice), leguminous c...
1                     NaN                                                NaN
2                     NaN                                                NaN
3                     NaN                                                NaN
4                     NaN                                                NaN
..                    ...                                                ...
410                   NaN                                                NaN
411                   NaN                                                NaN
412                   NaN                                                NaN
413                   NaN                                                NaN
414                   NaN                                                NaN

[415 rows x 2 columns]


In [66]:
keywords_of_class_i=df.loc[0,"Keywords_from_description"]
regex= r"[0-9]{4}"

re.findall(regex,keywords_of_class_i)


['0119']

In [104]:
import re
regex= r"[0-9]{4}"
i=0
while i< len(df.index):
    conn.query("Match(n:Class) return n.name")
    name_of_class=str(df.loc[i,"Name_of_class"])
    keywords_of_class_i=str(df.loc[i,"Keywords_from_description"])
    links_to_class_numbers= re.findall(regex,keywords_of_class_i)
    
    if len(links_to_class_numbers)!= 0:
        for link in links_to_class_numbers:
            if link.startswith("0"):
                link=link[1:]
                link=int(link)
                location=df_placeholder.index[df_placeholder["Placeholder_of_class"]==link]

                name=df_placeholder.loc[location,"Name"]
                name=str(list(name))
                name_of_class_to_link=name[2:-2]
                
                print("MATCH (n:Class) \nwhere n.name="+name_of_class+"\nMATCH (q:Class) \nwhere q.name='"+name_of_class_to_link+ "'\nCreate (n)-[:related_industries]->(q)-[:related_industries]->(n)")
            
            else:
                link=int(link)
                location=df_placeholder.index[df_placeholder["Placeholder_of_class"]==link]

                name=df_placeholder.loc[location,"Name"]
                name=str(list(name))
                name_of_class_to_link=name[2:-2]
                
                print("MATCH (n:Class) \nwhere n.name="+name_of_class+"\nMATCH (q:Class) \nwhere q.name='"+name_of_class_to_link+ "'\nCreate (n)-[:related_industries]->(q)-[:related_industries]->(n)")
            
    
    i=i+1

MATCH (n:Class) 
where n.name='Growing of cereals (except rice), leguminous crops and oil seeds'
MATCH (q:Class) 
where q.name='Growing of other non-perennial crop'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Growing of vegetables and melons, roots and tubers'
MATCH (q:Class) 
where q.name='Growing of spices, aromatic, drug and pharmaceutical crops'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Growing of vegetables and melons, roots and tubers'
MATCH (q:Class) 
where q.name='plant propagation'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Growing of sugar cane'
MATCH (q:Class) 
where q.name='Growing of vegetables and melons, roots and tubers'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Growing of other non-perennial crop'
MATCH (q:Class) 
where q.name='Growing of spices, aromatic, dru

MATCH (n:Class) 
where n.name='mining of other non-ferrous metal ores'
MATCH (q:Class) 
where q.name='manufacture of basic precious and other non-ferrous metals'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='mining of other non-ferrous metal ores'
MATCH (q:Class) 
where q.name='mining of uranium and thorium ores'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='quarrying of stone, sand and clay'
MATCH (q:Class) 
where q.name='Cutting, shaping and finishing of stone'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='quarrying of stone, sand and clay'
MATCH (q:Class) 
where q.name='manufacture of cement, lime and plaster'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='quarrying of stone, sand and clay'
MATCH (q:Class) 
where q.name='mining of chemical and fertilizer minerals'
Create (n)-[:related_in

MATCH (n:Class) 
where n.name='manufacture of bakery products'
MATCH (q:Class) 
where q.name='manufacture of macaroni, noodles, couscous and similar farinaceous'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of bakery products'
MATCH (q:Class) 
where q.name='processing and preserving of fruit and vegetables'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of sugar'
MATCH (q:Class) 
where q.name='manufacture of starches and starch products'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of cocoa, chocolate and sugar confectionery'
MATCH (q:Class) 
where q.name='manufacture of sugar'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of macaroni, noodles, couscous and similar farinaceous'
MATCH (q:Class) 
where q.name='manufacture of other food product

MATCH (n:Class) 
where n.name='manufacture of knitted and crocheted fabrics'
MATCH (q:Class) 
where q.name='manufacture of knitted and crocheted apparel'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of knitted and crocheted fabrics'
MATCH (q:Class) 
where q.name='manufacture of other textiles n.e.c.'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of made-up textile articles, except apparel'
MATCH (q:Class) 
where q.name='manufacture of other textiles n.e.c.'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of carpets and rugs'
MATCH (q:Class) 
where q.name=' manufacture of plastics products'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of carpets and rugs'
MATCH (q:Class) 
where q.name='manufacture of other products of wood; manufacture of arti

MATCH (n:Class) 
where n.name='manufacture of wooden containers'
MATCH (q:Class) 
where q.name='manufacture of other products of wood; manufacture of articles of cork,'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of wooden containers'
MATCH (q:Class) 
where q.name='manufacture of luggage, handbags and the like, saddlery and harness'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of other products of wood; manufacture of articles of cork,'
MATCH (q:Class) 
where q.name='other manufacturing n.e.c.'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of other products of wood; manufacture of articles of cork,'
MATCH (q:Class) 
where q.name='manufacture of watches and clocks'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='manufacture of other products of wood; man

...

MATCH (n:Class) 
where n.name='Site preparation'
MATCH (q:Class) 
where q.name=' architectural and engineering activities and related technical'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Site preparation'
MATCH (q:Class) 
where q.name='other specialized construction activities'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Site preparation'
MATCH (q:Class) 
where q.name='Construction of utility projects'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Site preparation'
MATCH (q:Class) 
where q.name='remediation activities and other waste management services'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Site preparation'
MATCH (q:Class) 
where q.name='extraction of crude petroleum'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Site prepar

MATCH (n:Class) 
where n.name='retail sale of computers, peripheral units, software and'
MATCH (q:Class) 
where q.name='retail sale of music and video recordings in specialized stores'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='retail sale of textiles in specialized stores'
MATCH (q:Class) 
where q.name='retail sale of clothing, footwear and leather articles in specialized'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Retail sale of carpets, rugs, wall and floor coverings in specialized'
MATCH (q:Class) 
where q.name='retail sale of hardware, paints and glass in specialized stores'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='retail sale of electrical household appliances, furniture, lighting'
MATCH (q:Class) 
where q.name='retail sale of second-hand goods'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Cla

MATCH (n:Class) 
where n.name='Cargo handling'
MATCH (q:Class) 
where q.name='Service activities incidental to land transportation'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Cargo handling'
MATCH (q:Class) 
where q.name='Service activities incidental to air transportation'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Cargo handling'
MATCH (q:Class) 
where q.name='Service activities incidental to water transportation'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Other transportation support activities'
MATCH (q:Class) 
where q.name=''
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Other transportation support activities'
MATCH (q:Class) 
where q.name=' tour operator activities'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Other transpo

MATCH (n:Class) 
where n.name='News agency activities'
MATCH (q:Class) 
where q.name=' Dramatic arts, music and other arts activities'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='News agency activities'
MATCH (q:Class) 
where q.name='photographic activities'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Other monetary intermediation'
MATCH (q:Class) 
where q.name=' Activities auxiliary to financial service activities n.e.c.'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='Other monetary intermediation'
MATCH (q:Class) 
where q.name='other credit granting'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='other credit granting'
MATCH (q:Class) 
where q.name='Other monetary intermediation'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name=' Security a

MATCH (n:Class) 
where n.name='other education n.e.c.'
MATCH (q:Class) 
where q.name='primary education'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='other education n.e.c.'
MATCH (q:Class) 
where q.name='higher education'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='other education n.e.c.'
MATCH (q:Class) 
where q.name='technical and vocational secondary education'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='other education n.e.c.'
MATCH (q:Class) 
where q.name='General secondary education'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='residential care activities for mental retardation, mental health and'
MATCH (q:Class) 
where q.name='other residential care activities n.e.c.'
Create (n)-[:related_industries]->(q)-[:related_industries]->(n)
MATCH (n:Class) 
where n.name='other residen

In [105]:
import re
regex= r"[0-9]{4}"
i=0
while i< len(df.index):
    conn.query("Match(n:Class) return n.name")
    name_of_class=str(df.loc[i,"Name_of_class"])
    keywords_of_class_i=str(df.loc[i,"Keywords_from_description"])
    links_to_class_numbers= re.findall(regex,keywords_of_class_i)
    
    if len(links_to_class_numbers)!= 0:
        for link in links_to_class_numbers:
            if link.startswith("0"):
                link=link[1:]
                link=int(link)
                location=df_placeholder.index[df_placeholder["Placeholder_of_class"]==link]

                name=df_placeholder.loc[location,"Name"]
                name=str(list(name))
                name_of_class_to_link=name[2:-2]
                
                conn.query("MATCH (n:Class) \nwhere n.name="+name_of_class+"\nMATCH (q:Class) \nwhere q.name='"+name_of_class_to_link+ "'\nCreate (n)-[:related_industries]->(q)-[:related_industries]->(n)")
            
            else:
                link=int(link)
                location=df_placeholder.index[df_placeholder["Placeholder_of_class"]==link]

                name=df_placeholder.loc[location,"Name"]
                name=str(list(name))
                name_of_class_to_link=name[2:-2]
                
                conn.query("MATCH (n:Class) \nwhere n.name="+name_of_class+"\nMATCH (q:Class) \nwhere q.name='"+name_of_class_to_link+ "'\nCreate (n)-[:related_industries]->(q)-[:related_industries]->(n)")
            
    
    i=i+1

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ',': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, USE GRAPH, LOAD CSV, START, MATCH, UNWIND, MERGE, CREATE UNIQUE, CREATE, SET, DELETE, REMOVE, FOREACH, WITH, CALL, RETURN, UNION, ';' or end of input (line 4, column 51 (offset: 150))
"where q.name='Raising of horses and other equines', 'manufacture of articles of fur'"
                                                   ^}


In [106]:
class_place_holder_to_name=[]
i=0
set_section=""
sum=0

f = open('Project0/code/neo4j_graph_database/Class_place_holder_name.csv', 'w')
writer = csv.writer(f)
row_1=["Placeholder_of_class","Name"]
writer.writerow(row_1)

while i<len(df_nic):
    
    i=i+1
    if re.search("^[0-9]+$", str(df_nic.loc[i,"Group"])) :
        temp_for_section ="Group_"+str(df_nic.loc[i,"Group"])
        set_section= temp_for_section 
        #print(set_section)
    
    description="""  """
        
  
        
    if re.search("^[0-9]+$", str(df_nic.loc[i,"Class"])) :
        temp_for_class =str(df_nic.loc[i,"Class"])
        
        j=i
        while str(df_nic.loc[j,"Sub-class"])=="nan" :
            #print(df.loc[j,"Description"])
            description = description+" " + str(df_nic.loc[j,"Description"])
            #print(description)
            j=j+1
        
        sum=sum+1
        temp=temp_for_class
        string=str(df_nic.loc[i,"Description"])
        #class_place_holder_to_name.append(set_section)
        #class_place_holder_to_name.append(description)
        temp=[temp_for_class,str(df_nic.loc[i,"Description"])]
        writer.writerow(temp)
        
        
      

FileNotFoundError: [Errno 2] No such file or directory: 'Project0/code/neo4j_graph_database/Class_place_holder_name.csv'